In [1]:
import numpy as np
from numba import jit, njit, vectorize, cfunc

In [2]:
N=10000000
#N=1000000
#N=100000
#N=10000
#N=1000
#N=100

In [3]:
a=np.random.rand(N)
b=np.random.rand(N)


Re=4000*np.exp(np.log(1e8/4000)*np.random.rand(N))
epsD=1e-6*np.exp(np.log(0.05/1e-6)*np.random.rand(N))


In [4]:
@njit(parallel=True)
def f_sum(Re,epsD):
    return Re+epsD

@njit(parallel=True)
def f_mult(Re,epsD):
    return Re*epsD

@jit(parallel=True)
def f_mult6(Re,epsD):
    for i in range(6):
        Re*epsD
    return Re*epsD

@jit(parallel=True)
def f_log(Re,epsD):  
    return np.log(Re)

#@jit(parallel=True)
def f_where(Re,epsD):  
    z=np.where(Re>1e5)*1+np.where(Re<1e5)*2

In [5]:
NN=16
name_=['' for i in range(NN+1)]
time_=np.zeros(NN+1)
MARE_=np.zeros(NN+1)

In [6]:
name_[0]='swamee jain'
@njit(parallel=True)
def f_0(Re,epsD): 
    z=np.log(epsD/3.7+5.74/np.exp(0.9*np.log(Re)))
    return 1.3254745276195998/(z*z) #0.25*np.power(np.log(10),2)

name_[1]='cw30'
@njit(parallel=True)
def f_1(Re,epsD):
    z=-8.6
    y1=epsD/3.7
    y2=-2.1801582991543236/Re   #(-2/np.log(10)*2.51)
    #manual unrolling 
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    return 1.3254745276195998/(z*z) #1/np.power((-2/np.log(10)),2)

name_[2]='cw5'
@njit(parallel=True)
def f_2(Re,epsD):
    z=-8.6
    y1=epsD/3.7
    y2=-2.1801582991543236/Re   #(-2/np.log(10)*2.51)
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    return 1.3254745276195998/(z*z) #1/np.power((-2/np.log(10)),2)


name_[3]='goudar-sonnad'
@jit(parallel=True)
def f_3(Re,epsD): #'Goudar–Sonnad'
    #
    a_=2/np.log(10)
    b_=epsD/3.7
    d_=np.log(10)*Re/5.02
    s_=b_*d_+np.log(d_)
    q_=np.exp(np.log(s_)*s_/(s_+1))
    #q_=np.power(s_,s_/(s_+1))
    g_=b_*d_+np.log(d_/q_)
    z_=np.log(q_/g_)
    dla_=z_*g_/(g_+1)
    dcfa_=dla_*(1+(z_/2)/((g_+1)*(g_+1)+z_/3*(2*g_-1)))
    f=a_*(np.log(d_/q_)+dcfa_)
    f=1/(f*f)
    #
    return f

name_[4]='haaland_1983'
@jit(parallel=True)
def f_4(Re,epsD): 
    f=np.log10(epsD/3.7+np.exp(np.log(7/Re)*0.9))
    f=0.25/(f*f)
    #
    return f

name_[5]='Barr1981'
@jit(parallel=True)
def f_5(Re,epsD): 
    y1=-2*np.log10(epsD/3.7+4.518*np.log10(Re/7)/(Re*(1+np.exp(np.log(Re)*0.52)/29*np.exp(np.log(epsD)*0.7))))
    f=1/(y1*y1)
    #
    return f

name_[6]='serghides1984'
@jit(parallel=True)
def f_6(Re,epsD): 
    y1=epsD/3.7
    y2=-0.8685889638065035*np.log(y1+12/Re)   #2/np.log(10)
    y3=-0.8685889638065035*np.log(y1+2.51*y2/Re)
    y4=-0.8685889638065035*np.log(y1+2.51*y3/Re)
    f=y2-(y3-y2)*(y3-y2)/(y4-2*y3+y2)
    #
    return 1/(f*f)

name_[7]='Vatankhah_K_08'
@jit(parallel=True)
def f_7(Re,epsD): 
    y1=0.124*Re*epsD+np.log(0.4587*Re)
    f=0.8686*np.log(0.4587*Re/(np.exp(np.log(y1-0.31)*y1/(y1+0.9633))))
    return 1/(f*f)

name_[8]='zig1982'
@jit(parallel=True)
def f_8(Re,epsD): 
    y1=epsD/3.7-5.02/Re*np.log10(epsD/3.7-5.02/Re*np.log10(epsD/3.7+13/Re))
    f=-0.8685889638065035*np.log(y1)
    f=1/(f*f)
    f=np.log10(2.51/(Re*np.sqrt(f))+epsD/3.7)
    return 0.25/(f*f)

name_[9]='S.Alashkar_2012'
@jit(parallel=True)
def f_9(Re,epsD): 
    y1=epsD/3.7065
    y2=2.5226/Re
    f=np.log(y1-0.8686068432*y2*np.log(y1-0.8784893582*y2*np.log(y1+np.exp(np.log(1.665368035*y2)*0.8373492157))))
    return 1.325474505/(f*f)

name_[10]='Fang 2011'
@jit(parallel=True)
def f_10(Re,epsD): 
    y1=0.234*np.exp(np.log(epsD)*1.1007)
    y2=-60.525*np.exp(np.log(Re)*-1.1105)
    y3=56.291*np.exp(np.log(Re)*-1.0712)
    f=np.log(y1+y2+y3)
    return 1.613/(f*f)

name_[11]='Evangelides'
@jit(parallel=True)
def f_11(Re,epsD): 
    y1=7-np.log10(Re)
    y1=y1*y1*y1*y1
    y1=0.2479-0.0000947*y1
    y2=np.log10(epsD/3.615+7.366*np.exp(np.log(Re)*-0.9142))
    return y1/(y2*y2)

name_[12]='vatan19'
@jit(parallel=True)
def f_12(Re,epsD): 
    y1=0.8685889638065035 #2/np.log(10)  -a
    y2=0.39840637450199207 #1/2.51  -a1
    y3=0.12363404028060514  #   a2
    y4=y3*Re*epsD+np.log(Re/2.51) #s
    y5=1+0.49959/((y4+1)/np.log(y1*y4)-(1+5.448*y4)/(8*(y4+1))) #lambda
    f=y1*np.log(y2*Re)-y1*y4/(y4+y5)*np.log(y1*y4)
    return 1/(f*f)

name_[13]='newton2'
@jit(parallel=True)
def f_13(Re,epsD): 
    z_=-6.880288946433447 #1/sqrt(0.028)*log(10)/(-2)
    y1=epsD/3.7
    y2=-2.180158299154324/Re     #-2/log(10)*2,51
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    return 1.325474527619600/(z_*z_)  #(ln(10)/(-2))^2

name_[14]='Newton_CW_5iter'
@jit(parallel=True)
def f_14(Re,epsD): 
    z_=-6.880288946433447 #1/sqrt(0.028)*log(10)/(-2)
    y1=epsD/3.7
    y2=-2.180158299154324/Re     #-2/log(10)*2,51
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    return 1.325474527619600/(z_*z_)  #(ln(10)/(-2))^2

name_[15]='Halley_CW_4iter'
@jit(parallel=True)
def f_15(Re,epsD): 
    z_=-6.880288946433447 #1/sqrt(0.028)*log(10)/(-2)
    y1=epsD/3.7
    y2=-2.180158299154324/Re #-2/log(10)*2,51
    f_=0
    temp=0
    fp_=0
    fpp_=0
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    return 1.325474527619600/(z_*z_)  #(ln(10)/(-2))^2
    
name_[16]='Praks'
@jit(parallel=True)
def f_16(Re,epsD): 
    a=-0.8685889638065035#(2/np.log(10))
    delta_=74205.5+1000*epsD*Re
    A=8-a*np.log(16/Re+epsD/3.7)
    B=-74914381.46/(delta_*delta_)
    C=1391459721232.67/(delta_*delta_*delta_)
    z_=8-A-0.5*A*A*B
    z_=a*np.log(2.51/Re*z_+epsD/3.7)
    z_=a*np.log(2.51/Re*z_+epsD/3.7)
    z_=a*np.log(2.51/Re*z_+epsD/3.7)
    #
    return 1/(z_*z_) #%(ln(10)/(-2))^2

In [7]:

def f_(Re,epsD,no_):
    if   no_==0:
        return f_0(Re,epsD)
    elif no_==1:
        return f_1(Re,epsD)
    elif no_==2:
        return f_2(Re,epsD)
    elif no_==3:
        return f_3(Re,epsD)
    elif no_==4:
        return f_4(Re,epsD)
    elif no_==5:
        return f_5(Re,epsD)
    elif no_==6:
        return f_6(Re,epsD)
    elif no_==7:
        return f_7(Re,epsD)
    elif no_==8:
        return f_8(Re,epsD)
    elif no_==9:
        return f_9(Re,epsD)
    elif no_==10:
        return f_10(Re,epsD)
    elif no_==11:
        return f_11(Re,epsD)
    elif no_==12:
        return f_12(Re,epsD)
    elif no_==13:
        return f_13(Re,epsD)
    elif no_==14:
        return f_14(Re,epsD)
    elif no_==15:
        return f_15(Re,epsD)
    elif no_==16:
        return f_16(Re,epsD)    

In [8]:
print('check accuracy with CW')
@njit(parallel=True)
def func_errCW(Re,epsD,z_):
    AA=1/np.sqrt(z_)+2/np.log(10)*np.log(epsD/3.7+2.51/(Re*np.sqrt(z_)))
    print('cw err=', np.max(np.abs(AA))*100,'%')

def func_err(z_,z_hat):
    return max(abs((z_-z_hat)/z_))*100

check accuracy with CW


In [9]:
z_=f_(Re,epsD,1)
func_errCW(Re,epsD,z_)

cw err= 3.552713678800501e-13 %


In [10]:
for i in range(NN+1):
    result = %timeit -o f_(Re,epsD,i)
    time_[i]=result.average
    MARE_[i]=func_err(z_,f_(Re,epsD,i))    


103 ms ± 8.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
919 ms ± 59.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
151 ms ± 5.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
173 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
125 ms ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
247 ms ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
116 ms ± 9.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
165 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
188 ms ± 24.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
164 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
191 ms ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
184 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
130 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
89.9 ms ± 8.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
205 ms ± 18 ms per loop (mean ± std. dev. of 7 ru

In [11]:
print('python-numba-numpy-intel 3610qm-N=',N)

for i in range(NN+1):
    print('%20s'%name_[i],'\t,t=\t',round(time_[i],6),'\ts,t/t_SJ=\t',round(time_[i]/time_[0],3),'\t,err=\t',MARE_[i],'\t%')
    


python-numba-numpy-intel 3610qm-N= 10000000
         swamee jain 	,t=	 0.10292 	s,t/t_SJ=	 1.0 	,err=	 3.358037925455073 	%
                cw30 	,t=	 0.918728 	s,t/t_SJ=	 8.927 	,err=	 0.0 	%
                 cw5 	,t=	 0.15105 	s,t/t_SJ=	 1.468 	,err=	 0.012977877765717082 	%
       goudar-sonnad 	,t=	 0.172981 	s,t/t_SJ=	 1.681 	,err=	 1.0368113138723236e-10 	%
        haaland_1983 	,t=	 0.12503 	s,t/t_SJ=	 1.215 	,err=	 3.418743204774515 	%
            Barr1981 	,t=	 0.247352 	s,t/t_SJ=	 2.403 	,err=	 0.5336528305845375 	%
       serghides1984 	,t=	 0.116101 	s,t/t_SJ=	 1.128 	,err=	 0.003099109818555796 	%
      Vatankhah_K_08 	,t=	 0.164643 	s,t/t_SJ=	 1.6 	,err=	 0.05154592909575923 	%
             zig1982 	,t=	 0.18758 	s,t/t_SJ=	 1.823 	,err=	 0.014312969411736008 	%
     S.Alashkar_2012 	,t=	 0.163789 	s,t/t_SJ=	 1.591 	,err=	 0.11916004696711836 	%
           Fang 2011 	,t=	 0.191498 	s,t/t_SJ=	 1.861 	,err=	 0.4907904155389456 	%
         Evangelides 	,t=	 0.184288 	s,t/t_SJ

In [12]:
#approximating log

#@jit
#def f(x):
    #y1=np.sqrt(x)
    #return -0.00911977367379447+0.0201791446960595*x-(0.92930823536411+0.0353105579811749*x+174.249927598629*y1)/(0.0584091431932963+184.459217046452*x+21.5475391442964*y1)
    #return -14.25054935+-139.9443694*y1+-0.001494028/y1+65.07628527*np.sqrt(y1)+302.9462386*x
    #return -0.00913491794126033+0.0201609704451216*x+-6.07070882259017*y1-(0.714558523718348+0.0353136870008399*x+191.855385668806*y1)/(0.0437599551230402+277.769273975508*x+21.6866445747455*y1)
    #xx=x/0.015752447
    #return -18.3320571697832+3.90142400857296*xx+(0.0931048343272263+91.2488786280689*xx)/(0.0189822790005925+8.0723529251094*xx)

    

In [190]:
@jit(parallel=True)
def f1(z):
    y1=np.logical_and(z<0.1    ,z>=0.01)
    y2=np.logical_and(z<0.01   ,z>=0.001)
    y3=np.logical_and(z<0.001  ,z>=0.0001)
    y4=np.logical_and(z<0.0001 ,z>=0.00001)
    y5=np.logical_and(z<0.00001,z>=0.000001)
    y6=np.logical_and(z<0.000001,z>=0.0000001) 
    z=-2*y1-3*y2-4*y3-5*y4-6*y5-7*y6
    zz=100*y1+1000*y2+10000*y3+100000*y4+1000000*y5+10000000*y6
    return z+np.log(zz)

def f5(z):
    y1=np.floor(np.log10(z))
    z=z/np.power(10,y1)
    return y1+np.log10(z)

@jit(parallel=True)
def f2(z):
    return np.log(z)

@jit(parallel=True)
def f3(z):
    y1=0.01*(z<0.01)
    return y1

@jit(parallel=True)
def f2(z):
    return np.log(z)


z1=np.float32(epsD/3.7-2.180158299154324/Re*(-6.8))
z2=np.float64(epsD/3.7-2.180158299154324/Re*(-6.8))

%timeit f1(z2)
%timeit f2(z2)



81.6 ms ± 7.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
35.7 ms ± 772 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [183]:
f1(0.00015)

-4

In [60]:
np.ldexp(5, Re)


TypeError: ufunc 'ldexp' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [62]:
np.log1p(10)

2.3978952727983707

In [63]:
np.log(10)


2.302585092994046

In [67]:
np.radians(500)

8.726646259971648

In [161]:
np.log(1.3,dtype='float32')

0.26236426446749106